<a href="https://colab.research.google.com/github/igorjoz/machine-learning-course-and-projects/blob/main/20_Bayes_game_tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

with open("tagi_gier_wspoldzielone.json", encoding="utf-8") as file:
  data = json.load(file)

# print(data)

label_count = {}

print(data)

for game in data:
  # print(game)
  label = game["label"] # np. minecraft
  if label not in label_count:
    label_count[label] = 0

  label_count[label] += 1

[{'label': 'minecraft', 'tags': ['rankingi', 'sandbox', 'pistolet', 'event', 'smok', 'diamenty', 'kolekcje', 'klocki']}, {'label': 'fortnite', 'tags': ['tryb solo', 'snajper', 'online', 'autobus bojowy', 'misje', 'questy', 'lama', 'fortyfikacje']}, {'label': 'fortnite', 'tags': ['sezon', 'kopanie', 'misje', 'drużyna', 'fortyfikacje', 'biomy', 'v-bucksy', 'lama']}, {'label': 'roblox', 'tags': ['czat', 'karabin', 'tryb kreatywny', 'klocki', 'oddziały', 'studio', 'misje', 'brokhaven']}, {'label': 'lol', 'tags': ['przetrwanie', 'snajper', 'zombie', 'esport', 'skiny', 'budowanie', 'ulti', 'dżungla']}, {'label': 'fortnite', 'tags': ['roleplay', 'online', 'slurp', 'lama', 'pistolet', 'symulator', 'tryb kreatywny', 'sandbox']}, {'label': 'fortnite', 'tags': ['kolekcje', 'karabin', 'autobus bojowy', 'biomy', 'lama', 'strzelanie', 'loot', 'symulator']}, {'label': 'roblox', 'tags': ['tryb', 'strzał w głowę', 'oddziały', 'questy', 'studio', 'duety', 'brokhaven', 'symulator']}, {'label': 'lol', 'ta

In [ ]:
import json, random, math

def load_data(path):
  with open(path, encoding="utf-8") as file:
    return json.load(file)

def train_test_split(data, test_ratio=0.2):
  random.shuffle(data)
  cut = int(len(data) * (1-test_ratio))
  print(f"Uczący zbiór: {len(data[:cut])}")
  print(f"Testowy zbiór: {len(data[cut:])}")
  return data[:cut], data[cut:]

def build_dictionary(train):
  vocabulary = set()
  for record in train:
    vocabulary.update(record["tags"])
  print(vocabulary)
  return vocabulary

def train_nb(train, vocabulary):
  class_counts = {}
  word_counts = {}
  total_words = {}

  for record in train:
    c = record["label"]
    class_counts[c] = class_counts.get(c, 0) + 1
    word_counts.setdefault(c, {})
    total_words.setdefault(c, 0)

    for tag in record["tags"]:
      word_counts[c][tag] = word_counts[c].get(tag, 0) + 1
      total_words[c] += 1

  model = {
      "class_counts": class_counts,
      "word_counts": word_counts,
      "total_words": total_words,
      "vocab": vocabulary,
      "alpha": 1.0,
      "total_docs": len(train)
  }

  print(model)
  return model

def log_prob(model, rec, class_name):
    logp = math.log(model["class_counts"][class_name] / model["total_docs"])
    V    = len(model["vocab"])
    a    = model["alpha"]


    for tag in rec["tags"]:
        wc = model["word_counts"][class_name].get(tag, 0)
        logp += math.log((wc + a) / (model["total_words"][class_name] + a * V))
    return logp

def predict(model, rec):
    best_class, best_log = None, -1e99
    for c in model["class_counts"]:
        lp = log_prob(model, rec, c)
        if lp > best_log:
            best_class, best_log = c, lp
    return best_class

def evaluate(model, test):
    correct = 0
    for rec in test:
        if predict(model, rec) == rec["label"]:
            correct += 1
    accuracy = correct / len(test)
    print(f"Dokładność (accuracy) = {accuracy:.2%}")

In [ ]:
def main():
  path = "tagi_gier_wspoldzielone.json"
  data = load_data(path)
  train, test = train_test_split(data)
  vocabulary = build_dictionary(train)
  model = train_nb(train, vocabulary)

  sample = test[0]
  print(sample["tags"])
  print("Prawda: " + sample["label"])
  print("Przewidzane: " + predict(model, sample))

  evaluate(model, test)

main()

Uczący zbiór: 240
Testowy zbiór: 60
{'nether', 'wydarzenia', 'strzelanie', 'przetrwanie', 'awp', 'obby', 'brokhaven', 'klocki', 'questy', 'smok', 'waluta', 'parkour', 'aktualizacja', 'symulator', 'tryb', 'roleplay', 'tryb solo', 'online', 'burza', 'v-bucksy', 'tryb kreatywny', 'loot', 'baron', 'event', 'robuxy', 'mapa', 'dżungla', 'survival', 'bomba', 'emotki', 'dust2', 'biomy', 'budowanie', 'pistolet', 'lama', 'esport', 'nexus', 'flash', 'sandbox', 'studio', 'hardcore', 'battle', 'tickrate', 'diamenty', 'redstone', 'prywatny serwer', 'strzał w głowę', 'karabin', 'drużyna', 'crafting', 'skiny', 'fortyfikacje', 'snajper', 'bohaterowie', 'multiplayer', 'oddziały', 'misje', 'sezon', 'czat', 'slurp', 'sklep', 'rankedy', 'ulti', 'rankingi', 'runy', 'zombie', 'kolekcje', 'duety', 'wyzwania', 'creepery', 'autobus bojowy', 'kopanie', 'arena', 'lua'}
{'class_counts': {'lol': 49, 'cs': 45, 'minecraft': 49, 'fortnite': 50, 'roblox': 47}, 'word_counts': {'lol': {'kopanie': 10, 'parkour': 3, 'nexus